In [ ]:
from pathlib import Path

voxel_sizes = [5, 2, 2] # in microns

In [ ]:
import pooch
data_path = Path.cwd().parent / "serial_2p_subset"
# Use pooch to fetch data if it hasn't already been downloaded
dsb_data_url = "https://gin.g-node.org/BrainGlobe/demo-materials/raw/master/serial2p_subset.zip"
data_path = pooch.retrieve(dsb_data_url, known_hash="c31e0136d00024ba74085baefbfd174c8e325f97a942c803ee720bdc8f6e7a00", progressbar=True, processor=pooch.Unzip(extract_dir=data_path))
data_path = Path(data_path[0]).parent.parent
signal_images = data_path / "ch00"
background_images = data_path / "ch02"

In [ ]:
import glob
import os

from dask import array as da
from dask import delayed
from imlib.general.system import get_sorted_file_paths
from tifffile import TiffFile, imread

In [ ]:
# Write a function to load data with dask
# Alternatively - `from cellfinder_core.tools.IO import read_with_dask`

lazy_imread = delayed(imread) 

def get_tiff_meta(path):
    with TiffFile(path) as tfile:
        nz = len(tfile.pages)
        if not nz:
            raise ValueError(f"tiff file {path} has no pages!")
        first_page = tfile.pages[0]

    return tfile.pages[0].shape, first_page.dtype


def read_with_dask(path):
    filenames = glob.glob(os.path.join(path, "*.tif"))
    shape, dtype = get_tiff_meta(filenames[0])
    lazy_arrays = [lazy_imread(fn) for fn in get_sorted_file_paths(filenames)]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=shape, dtype=dtype)
        for delayed_reader in lazy_arrays
    ]
    stack = da.stack(dask_arrays, axis=0)
    return stack

In [ ]:
from cellfinder_core.main import main as cellfinder_run

signal_array = read_with_dask(signal_images)
background_array = read_with_dask(background_images)

print(signal_array)
print(background_array)

In [ ]:
import napari
viewer = napari.view_image(background_array)
viewer.add_image(signal_array)

In [ ]:
detected_cells = cellfinder_run(signal_array,background_array,voxel_sizes)

In [ ]:
len(detected_cells)

In [ ]:
from cellfinder_napari.utils import add_layers as add_cell_layers
add_cell_layers(detected_cells, viewer)